# Chapter 5: How to get the data

In [2]:
import pandas as pd

## How to import a CSV file from a website

In [3]:
mortality_url = "https://data.cdc.gov/api/views/v6ab-adf5/rows.csv?accessType=DOWNLOAD"
mortality_data = pd.read_csv(mortality_url)

## How to import the first sheet of an Excel file

In [3]:
# NOTE: This code example is presented later in this Notebook, after the Excel file has been downloaded

## How to download a file to disk before importing it

In [4]:
from urllib import request
polls_url = \
    'http://projects.fivethirtyeight.com/general-model/president_general_polls_2016.csv'
request.urlretrieve(polls_url, filename='president_polls_2016.csv')

('president_polls_2016.csv', <http.client.HTTPMessage at 0x241aca67850>)

In [5]:
polls = pd.read_csv('president_polls_2016.csv')
polls.head(2)

,cycle,branch,type,matchup,forecastdate,state,startdate,enddate,pollster,grade,...,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin,multiversions,url,poll_id,question_id,createddate,timestamp
0,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/3/2016,11/6/2016,ABC News/Washington Post,A+,...,45.20163,41.72430,4.626221,NaN,NaN,https://www.washingtonpost.com/news/the-fix/wp...,48630,76192,11/7/16,09:35:33 8 Nov 2016
1,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/1/2016,11/7/2016,Google Consumer Surveys,B,...,43.34557,41.21439,5.175792,NaN,NaN,https://datastudio.google.com/u/0/#/org//repor...,48847,76443,11/7/16,09:35:33 8 Nov 2016


## How to work with a zip file on disk

### Download the zip file to disk

In [6]:
# NOTE: This code may take a long time to run
from urllib import request
zip_url = 'https://www.bls.gov/oes/special.requests/oesm18all.zip'
request.urlretrieve(zip_url, filename='oesm18all.zip')

('oesm18all.zip', <http.client.HTTPMessage at 0x241acaad700>)

### Extract the files and list their names

In [7]:
from zipfile import ZipFile
file_names = list() 
with ZipFile('oesm18all.zip', mode='r') as zip:
    zip.extractall()
    for file in zip.infolist():
        file_names.append(file.filename)
        print(file.filename, file.compress_size, file.file_size)

oesm18all/all_data_M_2018.xlsx 70296790 71834374


### Two ways to read an extracted file into a DataFrame

In [8]:
# NOTE: This code may take a long time to run
jobs = pd.read_excel("oesm18all/all_data_M_2018.xlsx")

In [9]:
# NOTE: This code may take a long time to run
jobs = pd.read_excel(file_names[0])

## How to run queries against a database

In [10]:
import sqlite3
fires_con = sqlite3.connect('../../data/Data/FPA_FOD_20170508.sqlite')
fires_cur = fires_con.cursor()

In [11]:
fires_cur.execute(
    'SELECT name FROM sqlite_master WHERE type="table"').fetchall()

[('spatial_ref_sys',),
 ('spatialite_history',),
 ('sqlite_sequence',),
 ('geometry_columns',),
 ('spatial_ref_sys_aux',),
 ('views_geometry_columns',),
 ('virts_geometry_columns',),
 ('geometry_columns_statistics',),
 ('views_geometry_columns_statistics',),
 ('virts_geometry_columns_statistics',),
 ('geometry_columns_field_infos',),
 ('views_geometry_columns_field_infos',),
 ('virts_geometry_columns_field_infos',),
 ('geometry_columns_time',),
 ('geometry_columns_auth',),
 ('views_geometry_columns_auth',),
 ('virts_geometry_columns_auth',),
 ('sql_statements_log',),
 ('SpatialIndex',),
 ('ElementaryGeometries',),
 ('KNN',),
 ('Fires',),
 ('idx_Fires_Shape',),
 ('idx_Fires_Shape_node',),
 ('idx_Fires_Shape_rowid',),
 ('idx_Fires_Shape_parent',),
 ('NWCG_UnitIDActive_20170109',)]

## How to use a SQL query to import data into a DataFrame

### Get information about a table

In [12]:
fires_cur.execute('PRAGMA table_info(Fires)').fetchall()

[(0, 'OBJECTID', 'integer', 1, None, 1),
 (1, 'FOD_ID', 'int32', 0, None, 0),
 (2, 'FPA_ID', 'text(100)', 0, None, 0),
 (3, 'SOURCE_SYSTEM_TYPE', 'text(255)', 0, None, 0),
 (4, 'SOURCE_SYSTEM', 'text(30)', 0, None, 0),
 (5, 'NWCG_REPORTING_AGENCY', 'text(255)', 0, None, 0),
 (6, 'NWCG_REPORTING_UNIT_ID', 'text(255)', 0, None, 0),
 (7, 'NWCG_REPORTING_UNIT_NAME', 'text(255)', 0, None, 0),
 (8, 'SOURCE_REPORTING_UNIT', 'text(30)', 0, None, 0),
 (9, 'SOURCE_REPORTING_UNIT_NAME', 'text(255)', 0, None, 0),
 (10, 'LOCAL_FIRE_REPORT_ID', 'text(255)', 0, None, 0),
 (11, 'LOCAL_INCIDENT_ID', 'text(255)', 0, None, 0),
 (12, 'FIRE_CODE', 'text(10)', 0, None, 0),
 (13, 'FIRE_NAME', 'text(255)', 0, None, 0),
 (14, 'ICS_209_INCIDENT_NUMBER', 'text(255)', 0, None, 0),
 (15, 'ICS_209_NAME', 'text(255)', 0, None, 0),
 (16, 'MTBS_ID', 'text(255)', 0, None, 0),
 (17, 'MTBS_FIRE_NAME', 'text(50)', 0, None, 0),
 (18, 'COMPLEX_NAME', 'text(255)', 0, None, 0),
 (19, 'FIRE_YEAR', 'int16', 0, None, 0),
 (20, '

### Import the data from a query into a DataFrame

In [13]:
# NOTE: This code may take a long time to run
fires = pd.read_sql_query(
    '''SELECT STATE, FIRE_YEAR, DATETIME(DISCOVERY_DATE) AS DISCOVERY_DATE,
       FIRE_NAME, FIRE_SIZE, LATITUDE, LONGITUDE
    FROM Fires''', fires_con)
fires.head()

,STATE,FIRE_YEAR,DISCOVERY_DATE,FIRE_NAME,FIRE_SIZE,LATITUDE,LONGITUDE
0,CA,2005,2005-02-02 00:00:00,FOUNTAIN,0.10,40.036944,-121.005833
1,CA,2004,2004-05-12 00:00:00,PIGEON,0.25,38.933056,-120.404444
2,CA,2004,2004-05-31 00:00:00,SLACK,0.10,38.984167,-120.735556
3,CA,2004,2004-06-28 00:00:00,DEER,0.10,38.559167,-119.913333
4,CA,2004,2004-06-28 00:00:00,STEVENOT,0.10,38.559167,-119.933056


## How to build DataFrames for the metadata and the data in a Stata file

### Get metadata from a Stata file

In [14]:
import pyreadstat
gss_stata_filename = '../../data/GSS7218_R3.DTA'
gss_empty, gss_meta = pyreadstat.read_dta(
    gss_stata_filename,
    metadataonly=True)
type(gss_meta)

pyreadstat._readstat_parser.metadata_container

### What the attributes of the metadata container can tell you

In [15]:
print("Number of columns:", gss_meta.number_columns)
print("Number of rows:", gss_meta.number_rows)
print("Column names:", gss_meta.column_names)

Number of columns: 6110
Number of rows: 64814
Column names: ['year', 'id', 'wrkstat', 'hrs1', 'hrs2', 'evwork', 'occ', 'prestige', 'wrkslf', 'wrkgovt', 'commute', 'industry', 'occ80', 'prestg80', 'indus80', 'indus07', 'occonet', 'found', 'occ10', 'occindv', 'occstatus', 'occtag', 'prestg10', 'prestg105plus', 'indus10', 'indstatus', 'indtag', 'marital', 'martype', 'agewed', 'divorce', 'widowed', 'spwrksta', 'sphrs1', 'sphrs2', 'spevwork', 'cowrksta', 'cowrkslf', 'coevwork', 'cohrs1', 'cohrs2', 'spocc', 'sppres', 'spwrkslf', 'spind', 'spocc80', 'sppres80', 'spind80', 'spocc10', 'spoccindv', 'spoccstatus', 'spocctag', 'sppres10', 'sppres105plus', 'spind10', 'spindstatus', 'spindtag', 'coocc10', 'coind10', 'paocc16', 'papres16', 'pawrkslf', 'paind16', 'paocc80', 'papres80', 'paind80', 'paocc10', 'paoccindv', 'paoccstatus', 'paocctag', 'papres10', 'papres105plus', 'paind10', 'paindstatus', 'paindtag', 'maocc80', 'mapres80', 'mawrkslf', 'maind80', 'maocc10', 'maoccindv', 'maoccstatus', 'maoc

### How to build a DataFrame for the column descriptions in the metadata

In [16]:
meta_cols=pd.DataFrame(
    data=gss_meta.column_labels,
    index=gss_meta.column_names,
    columns=['description'])
meta_cols.head(5)

,description
year,gss year for this respondent
id,respondent id number
wrkstat,labor force status
hrs1,number of hours worked last week
hrs2,number of hours usually work a week


### How to import columns of the data into a DataFrame

In [17]:
gss_data = pd.read_stata('../../data/GSS7218_R3.DTA',
    columns=['year','id','wrkstat','hrs1','hrs2','evwork','wrkslf','wrkgovt'])
gss_data.tail()

,year,id,wrkstat,hrs1,hrs2,evwork,wrkslf,wrkgovt
64809,2018,2344,working fulltime,36.0,NaN,NaN,someone else,government
64810,2018,2345,working parttime,36.0,NaN,NaN,someone else,private
64811,2018,2346,retired,NaN,NaN,yes,someone else,private
64812,2018,2347,retired,NaN,NaN,yes,someone else,private
64813,2018,2348,keeping house,NaN,NaN,yes,someone else,government


## How to download a JSON file

In [18]:
import json
from urllib import request
shots_url = 'https://www.murach.com/python_analysis/shots.json'
request.urlretrieve(shots_url, filename='shots.json')

('shots.json', <http.client.HTTPMessage at 0x241ae595850>)

## How to build a DataFrame for the data

In [19]:
with open('shots.json') as json_data:
    shots = json.load(json_data)

In [20]:
allRows = shots['resultSets'][0]['rowSet']
columnHeaders = [x.lower() for x in shots['resultSets'][0]['headers']]
shots = pd.DataFrame(data=allRows, columns=columnHeaders)
shots.head()

,grid_type,game_id,game_event_id,player_id,player_name,team_id,team_name,period,minutes_remaining,seconds_remaining,...,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_attempted_flag,shot_made_flag,game_date,htm,vtm
0,Shot Chart Detail,0020900015,4,201939,Stephen Curry,1610612744,Golden State Warriors,1,11,25,...,Right Side Center(RC),24+ ft.,26,99,249,1,0,20091028,GSW,HOU
1,Shot Chart Detail,0020900015,17,201939,Stephen Curry,1610612744,Golden State Warriors,1,9,31,...,Left Side Center(LC),16-24 ft.,18,-122,145,1,1,20091028,GSW,HOU
2,Shot Chart Detail,0020900015,53,201939,Stephen Curry,1610612744,Golden State Warriors,1,6,2,...,Center(C),8-16 ft.,14,-60,129,1,0,20091028,GSW,HOU
3,Shot Chart Detail,0020900015,141,201939,Stephen Curry,1610612744,Golden State Warriors,2,9,49,...,Left Side(L),16-24 ft.,19,-172,82,1,0,20091028,GSW,HOU
4,Shot Chart Detail,0020900015,249,201939,Stephen Curry,1610612744,Golden State Warriors,2,2,19,...,Left Side Center(LC),16-24 ft.,16,-68,148,1,0,20091028,GSW,HOU
